# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pandas.io.json import json_normalize
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
cities_df = pd.read_csv("cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Lalmohan,74,BD,2020-01-28 23:18:02,56,22.34,90.74,71.31,3.09
1,1,Pochutla,20,MX,2020-01-28 23:18:03,74,15.74,-96.47,78.80,11.39
2,2,Mataura,96,NZ,2020-01-28 23:18:03,89,-46.19,168.86,55.00,11.99
3,3,Thompson,90,CA,2020-01-28 23:15:49,85,55.74,-97.86,10.40,5.82
4,4,Severo-Kuril'sk,100,RU,2020-01-28 23:18:03,85,50.68,156.12,25.90,3.33


# Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
locations = cities_df[['Lat', 'Lng']]
humidity = cities_df['Humidity']
fig = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, point_radius = 10))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#* A max temperature lower than 80 degrees but higher than 70.
#* Wind speed less than 10 mph.
#* Zero cloudiness.
cities_df=cities_df.dropna(how="all")
new_df1 = cities_df[cities_df["Wind Speed"] < 10]
new_df2 = new_df1[new_df1["Cloudiness"] == 0]
new_df3 = cities_df[cities_df["Max Temp"] < 80]
fav_df = new_df3[new_df3["Max Temp"] > 70]
fav_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Lalmohan,74,BD,2020-01-28 23:18:02,56,22.34,90.74,71.31,3.09
1,1,Pochutla,20,MX,2020-01-28 23:18:03,74,15.74,-96.47,78.80,11.39
6,6,Rikitea,100,PF,2020-01-28 23:18:03,78,-23.12,-134.97,79.79,6.22
8,8,Hilo,20,US,2020-01-28 23:18:03,57,19.73,-155.09,78.80,5.82
13,13,Talara,0,PE,2020-01-28 23:18:04,87,-4.58,-81.27,74.30,9.80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = fav_df
HotelName = []
HotelAddress=[]

def Hoteldetails(Lat,Lng):
    TgtRadius = 5000
    TgtType = "Hotel"
    TgtCoordinates =  Lat + " , " + Lng 
    params = {
    "location": TgtCoordinates,
    "radius": TgtRadius,
    "type": TgtType,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    resp = requests.get(base_url, params=params)
    rtext = json.loads(resp.text)
    response = json_normalize(rtext)
    ResultedHotels=json_normalize(response["results"][0])
    HotelName.append(ResultedHotels["name"][1])
    HotelAddress.append(ResultedHotels["vicinity"][1])

In [ ]:
for index, row in fav_df.iterrows():
    Hoteldetails(str(row["Lat"]),str(row["Lng"]))

In [ ]:
HotelName,HotelAddress

In [ ]:
hotel_df.insert(2, "Hotel Name", HotelName, True) 
hotel_df.insert(3, "Hotel Address", HotelAddress, True)

In [13]:
# Create a Hotels
# Store 'Lat' and 'Lng' into  locations 
hlocations = hotel_df[["Lat", "Lng"]].astype(float)
HumidityValue = cities_df["Humidity"].astype(float)
maxHumidity=cities_df["Humidity"].max()

fig2 = gmaps.figure(center=(20.0, 0.0), zoom_level=2)
fig2.add_layer(gmaps.heatmap_layer(locations, weights=HumidityValue, dissipating=False, max_intensity=maxHumidity, point_radius = 2))
fig2

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map